## Recap

In the previous notebook, we did the following:-
* Built a block --> class
* Built a sample hashing/mining technique

* Synced with a single node

In [33]:
#Move this to a configuration file --> config.py

CHAINDATA_DIR = 'chaindata/'

# Use os.path.join to do this --> makes it os agnostic
BROADCASTED_BLOCK_DIR = CHAINDATA_DIR + 'bblocs/'

BLOCK_VAL_CONVERSIONS = {
    'index' : int,
    'nonce' : int,
    'hash'  : str,
    'prev_hash' : str,
    'timestamp' : int
}

NUM_ZEROS = 5 #difficulty

### Utils

Let's 

### Let's recreate the block
All in one place, unlike the previous one

In [89]:
## this would go to block.py
# from config import *

class Block(object):
    # We want the block to accept anything, 
    # but only hash what we want it to hash
    def __init__(self, *args, **kwargs):
        for key,value in kwargs.items():
            if key in BLOCK_VAL_CONVERSIONS:
                # Shows how each of them should be converted
                # Thank you config
                setattr(self, key, BLOCK_VAL_CONVERSIONS[key](value))
            else:
                setattr(self, key,value) # No conversion if it doesn't follow template
    
        # Calculate hash automatically
        # if not added, calculate it --> for the first block ?
        if not hasattr(self, 'hash'):
            self.hash = self.create_self_hash()
        
        if not hasattr(self, 'nonce'):
            self.nonce = 'None'
            
    def self_save(self):
        index_string = str(self.index).zfill(6)
        filename = '%s%s.json' % (CHAINDATA_DIR, index_string)
        with open(filename, 'w') as block_file:
            json.dump(self.to_dict(), block_file)
    
    def create_self_hash(self):
        return "hash placeholder"
    
    # check if the hash is valid.
    # Checks if there are NUM_ZEROS number of leading zeros
    def is_valid(self):
        self.update_self_hash()
        if str(self.hash[0:NUM_ZEROS]) == '0' * NUM_ZEROS:
            return True
        else:
            return False
        
    def __repr__(self):
        return "Block<index: %s>, <hash: %s>" % (self.index, self.hash)
    
    def __eq__(self,other):
        print (self.index, other.index)
        return(
            self.index == other.index and
            self.timestamp == other.timestamp and
            self.prev_hash == other.prev_hash and
            self.hash == other.hash and
            self.data == other.data and
            self.nonce == other.nonce
        )
    
    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __dict__(self):
        info = {}
        info['index'] = str(self.index)
        info['timestamp'] = str(self.timestamp)
        info['prev_hash'] = str(self.prev_hash)
        info['hash'] = str(self.hash)
        info['data'] = str(self.data)
        info['nonce'] = str(nonce)
        return info
    
    def update_self_hash(self):
        return self.hash

    
import datetime

def create_first_block():
    block_data = {}
    block_data['index'] = 0
    block_data['timestamp'] = datetime.datetime.now()
    block_data['data'] = 'First block data'
    block_data['prev_hash'] = None
    block = Block(block_data)
    return block
new_block = create_first_block()

###  Chains
Time to chain blocks to gether to create chains

In [103]:
# this goes to chains.py --> probably
# from block import Block


class Chain(object):
    
    def __init__(self, blocks):
        self.blocks = blocks
        for bla in blocks:
            print(bla.index)
    
    def is_valid(self):
        '''
        Got to check if chain is valid. By :-
        1) Each block is indexed in order
        2) prev_hash of each block = hash of previous block
        3) block's hash is valid
        '''
        for index, cur_block in enumerate(self.blocks[1:]):
            print (index)
            prev_block = self.blocks[index-1]
            cur_block = self.blocks[index]
            print (prev_block, cur_block)


            # Check all three conditions
            print (prev_block.index, cur_block.index)

            if prev_block.index+1 != cur_block.index:
                print ("Index dont match")
                return False
            if not cur_block.is_valid():
                return False
            if prev_block.hash != cur_block.prev_hash:
                print (prev_block.hash, cur_block.hash)
                return False
        return True
    
    def self_save(self):
        for b in self.blocks:
            b.self_save()
        return True
    
    #WTF is this function doing?
    def find_block_by_index(self, index):
        #Can't return a block outside the list of blocks
        if len(self.blocks) <= index:
            return self.blocks[index]
        else:
            return False
        
    def find_block_by_hash(self, hash):
        for b in self.blocks:
            if b.hash == hash:
                return b
        return False
    
    # Some operators
    def __len__(self):
        return len(self.blocks)
    
    def __eq__(self, other):
        if len(self.blocks) != len(other.blocks):
            return False
        for self_block, other_block in zip(self.blocks, other.blocks):
            if self_block != other_block:
                return False

    def __gt__(self, other):
        return len(self.blocks) > len(other.blocks)
    
    def __ge__(self, other):
        return self.__eq__(other) or self.__gt__(other)
    
    
    def add_block(self, new_block):
        # Have to add
        # Only add if hash is valid
        # Index is valid
        # else return False
        self.blocks.append(new_block)
        return True
    
    def block_list_dict(self):
        return [b.to_dict() for b in self.blocks]

### Testing
Time to test some stuff out. Just some basic testing. Just going to instantiate the chain.

In [95]:
# from block import Block
# from chain import Chain


block_zero_dir = {"nonce": "631412", "index": "0", "hash": "000002f9c703dc80340c08462a0d6acdac9d0e10eb4190f6e57af6bb0850d03c", "timestamp": "1508895381", "prev_hash": "", "data": "First block data"}
block_one_dir = {"nonce": "1225518", "index": "1", "hash": "00000c575050241e0a4df1acd7e6fb90cc1f599e2cc2908ec8225e10915006cc", "timestamp": "1508895386", "prev_hash": "000002f9c703dc80340c08462a0d6acdac9d0e10eb4190f6e57af6bb0850d03c", "data": "I block #1"}
block_two_dir = {"nonce": "1315081", "index": "2", "hash": "000003cf81f6b17e60ef1e3d8d24793450aecaf65cbe95086a29c1e48a5043b1", "timestamp": "1508895393", "prev_hash": "00000c575050241e0a4df1acd7e6fb90cc1f599e2cc2908ec8225e10915006cc", "data": "I block #2"}
block_three_dir = {"nonce": "24959", "index": "3", "hash": "00000221653e89d7b04704d4690abcf83fdb144106bb0453683c8183253fabad", "timestamp": "1508895777", "prev_hash": "000003cf81f6b17e60ef1e3d8d24793450aecaf65cbe95086a29c1e48a5043b1", "data": "I block #3"}
block_three_later_in_time_dir = {"nonce": "46053", "index": "3", "hash": "000000257df186344486c2c3c1ebaa159e812ca1c5c29947651672e2588efe1e", "timestamp": "1508961173", "prev_hash": "000003cf81f6b17e60ef1e3d8d24793450aecaf65cbe95086a29c1e48a5043b1", "data": "I block #3"}


# 
# Let's test some blocks


block_zero  = Block(**block_zero_dir)
another_block_zero = Block(**block_zero_dir)
print(block_zero.is_valid())
assert block_zero.is_valid()
assert block_zero == another_block_zero
assert not block_zero != another_block_zero

block_one = Block(**block_one_dir)
block_two = Block(**block_two_dir)

True
0 0
0 0


In [104]:

blockchain = Chain([block_zero, block_one, block_two])
print (blockchain.is_valid())
# assert blockchain.is_valid()

0
1
2
0
Block<index: 2>, <hash: 000003cf81f6b17e60ef1e3d8d24793450aecaf65cbe95086a29c1e48a5043b1> Block<index: 0>, <hash: 000002f9c703dc80340c08462a0d6acdac9d0e10eb4190f6e57af6bb0850d03c>
2 0
Index dont match
False
